In [1]:
import pickle
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
import torch
from time import time

C:\Users\EXTREME_RED\AppData\Local\Temp\ipykernel_16264\2035421041.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [27]:
import json


def read_json_file(json_file_path):
    descriptions = {}
    try:
        with open(json_file_path, 'r') as file:
            data = json.load(file)
            for item in data:
                if 'image_id' in item and 'paragraph' in item:
                    image_id_str = str(item['image_id'])
                    descriptions[image_id_str] = item['paragraph']
                else:
                    print("Invalid format: Missing 'image_id' or 'paragraph' key in JSON item.")
    except FileNotFoundError:
        print("File not found.")
    except json.JSONDecodeError:
        print("Invalid JSON format in file.")
    except Exception as e:
        print("An error occurred:", e)
    return descriptions


json_file_path = 'paragraphs_v1.json'  # Specify the path to your JSON file
descriptions = read_json_file(json_file_path)
#print("Descriptions:", descriptions)
print('Loaded: %d ' % len(descriptions))

Loaded: 19551 


In [28]:
list(descriptions.keys())[:5]

['2356347', '2317429', '2414610', '2365091', '2383120']

In [29]:
count = 0
for image_id, paragraph in descriptions.items():
    print(f"Image ID: {image_id}, Paragraph: {paragraph}")
    count += 1
    if count == 5:
        break

Image ID: 2356347, Paragraph: A large building with bars on the windows in front of it. There is people walking in front of the building. There is a street in front of the building with many cars on it. 
Image ID: 2317429, Paragraph: A white round plate is on a table with a plastic tablecloth on it. Two foil covered food halves are on the white plate along with a serving of golden yellow french fries. Next to the white plate in a short, topless, plastic container is a white sauce. Diagonal to the white plate are the edges of several other stacked plates. There are black shadows reflected on the table.
Image ID: 2414610, Paragraph: A woman in a blue tennis outfit stands on a green tennis court. She is swinging a blue tennis racket. There is a green tennis ball above her head. 
Image ID: 2365091, Paragraph: A large red and white train is traveling on tracks in a what looks to be a rural area. There are trees and hills in the background and the ground looks dry. The train has many large w

In [30]:
print(descriptions['2356347'])

A large building with bars on the windows in front of it. There is people walking in front of the building. There is a street in front of the building with many cars on it. 


In [32]:
import string

def clean_descriptions(descriptions):
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for key, desc in descriptions.items():
        # tokenize
        desc = desc.split()
        # convert to lower case
        desc = [word.lower() for word in desc]
        # remove punctuation from each token
        desc = [w.translate(table) for w in desc]
        # remove hanging 's' and 'a'
        desc = [word for word in desc if len(word)>1]
        # remove tokens with numbers in them
        desc = [word for word in desc if word.isalpha()]
        # store as string
        descriptions[key] = ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)

In [34]:
descriptions['2414610']

'woman in blue tennis outfit stands on green tennis court she is swinging blue tennis racket there is green tennis ball above her head'

In [37]:
def create_vocabulary(descriptions):
    vocabulary = set()
    for key, desc in descriptions.items():
        # Tokenize the paragraph into words
        words = desc.split()
        # Add unique words to the vocabulary set
        vocabulary.update(words)
    return vocabulary

# Create vocabulary
vocabulary = create_vocabulary(descriptions)
#print("Vocabulary:", vocabulary)
print("Vocabulary Size:", len(vocabulary))


Vocabulary Size: 13449


In [39]:
def save_descriptions_to_file(descriptions, file_path):
    with open(file_path, 'w') as file:
        for key, desc in descriptions.items():
            file.write(f"{key} {desc}\n")


file_path = 'descriptions.txt'  
save_descriptions_to_file(descriptions, file_path)


In [41]:
import json

def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print("File not found.")
        return {}
    except json.JSONDecodeError:
        print("Invalid JSON format in file.")
        return {}
    except Exception as e:
        print("An error occurred:", e)
        return {}

def save_descriptions_to_file(descriptions, file_path):
    with open(file_path, 'w') as file:
        for key, desc in descriptions.items():
            file.write(f"{key} {desc}\n")

# Load train_images.json file
test_images_file_path = 'train_split.json'
with open(test_images_file_path, 'r') as file:
    test_images_data = json.load(file)

# Convert image IDs to strings
image_ids = [str(image_id) for image_id in test_images_data]

# Filter descriptions dictionary to include only image IDs present in train_images.json
filtered_descriptions = {key: desc for key, desc in descriptions.items() if key in image_ids}

# Save filtered descriptions to a text file
file_path = 'train_descriptions.txt'
save_descriptions_to_file(filtered_descriptions, file_path)
#print("Filtered descriptions saved to", file_path)



In [42]:
# Load test_images.json file
test_images_file_path = 'test_split.json'
with open(test_images_file_path, 'r') as file:
    test_images_data = json.load(file)

# Convert image IDs to strings
image_ids = [str(image_id) for image_id in test_images_data]

# Filter descriptions dictionary to include only image IDs present in test_images.json
filtered_descriptions = {key: desc for key, desc in descriptions.items() if key in image_ids}
print(len(filtered_descriptions))
# Save filtered descriptions to a text file
file_path = 'test_descriptions.txt'
save_descriptions_to_file(filtered_descriptions, file_path)

2489


In [43]:
# Load val_images.json file
test_images_file_path = 'val_split.json'
with open(test_images_file_path, 'r') as file:
    test_images_data = json.load(file)

# Convert image IDs to strings
image_ids = [str(image_id) for image_id in test_images_data]

# Filter descriptions dictionary to include only image IDs present in val_images.json
filtered_descriptions = {key: desc for key, desc in descriptions.items() if key in image_ids}
print(len(filtered_descriptions))
# Save filtered descriptions to a text file
file_path = 'val_descriptions.txt'
save_descriptions_to_file(filtered_descriptions, file_path)

2487


In [9]:
import json
import torchtext
from torchtext.vocab import GloVe

# Function to preprocess a single description
def preprocess_description(description, tokenizer, stopwords):
    tokenized = tokenizer(description.lower())
    tokenized = [token for token in tokenized if token.isalpha()]
    tokenized = [token for token in tokenized if token not in stopwords]
    tokenized = [token if not token.isdigit() else "<NUM>" for token in tokenized]
    encoded = [glove[token].tolist() for token in tokenized if token in glove.stoi]
    return encoded

# Sample descriptions file format: image_name description
descriptions_file = "filtered_test_descriptions.txt"
preprocessed_data = {}

# Load GloVe embeddings
glove = GloVe(name='6B', dim=100)
stopwords = set(' '.join(torchtext.data.utils.get_tokenizer("basic_english")('')).split())
tokenizer = torchtext.data.utils.get_tokenizer("basic_english")

# Read descriptions file line by line
with open(descriptions_file, 'r') as file:
    for line in file:
        image_name, description = line.strip().split(maxsplit=1)
        preprocessed_data[image_name] = preprocess_description(description, tokenizer, stopwords)

# Save preprocessed data to JSON file
output_file = "preprocessed_filtered_test_descriptions.json"
with open(output_file, 'w') as file:
    json.dump(preprocessed_data, file)

print("Preprocessed descriptions saved to:", output_file)


Preprocessed descriptions saved to: preprocessed_filtered_test_descriptions.json


In [16]:
import torch
import json
from torchtext.vocab import GloVe

# Load GloVe embeddings
glove = GloVe(name='6B', dim=200)

# Function to decode encoded descriptions
def decode_description(encoded_description):
    decoded_tokens = []
    for embedding in encoded_description:
        # Find the nearest word in GloVe embeddings
        embedding_tensor = torch.tensor(embedding)
        distances = torch.norm(glove.vectors - embedding_tensor, dim=1)
        nearest_index = torch.argmin(distances).item()
        nearest_word = glove.itos[nearest_index]
        decoded_tokens.append(nearest_word)
    return ' '.join(decoded_tokens)

# Load preprocessed data from JSON file
preprocessed_file = "preprocessed_filtered_train_descriptions.json"
with open(preprocessed_file, 'r') as file:
    preprocessed_data = json.load(file)

# Decode and print descriptions
for image_name, encoded_description in preprocessed_data.items():
    decoded_description = decode_description(encoded_description)
    print(f"Image: {image_name}, Decoded Description: {decoded_description}")


Image: 2401672, Decoded Description: yellow wicker and metal chair sits in corner of deck on boat there are backpacks along the side of the wooden deck shadows are casted on the floor rope ladder is to the left of the deck water surrounds the boat and island land masses are visible in the distance
Image: 2378834, Decoded Description: group of people are at the beach two women are sitting in blue beach chairs they have blonde hair pink shirts shorts and are wearing sunglasses man is sitting behind them at picnic table in denim pants and blue and white stripped shirts shirtless boy is pushing bike along side young man four small children play near the shore the two boys are shirtless and wearing shorts and the girls are wearing bathing suits
Image: 2349091, Decoded Description: two brown horses are standing together outside there heads are up against the side of the building the both have saddles on they are standing on concrete
Image: 2349250, Decoded Description: this is picture of an 

KeyboardInterrupt: 

In [16]:
import torchtext
import torchtext.data

descriptions_file = "filtered_test_descriptions.txt"

counter = torchtext.vocab.Counter()
max_length = 0  # Initialize max_length variable
with open(descriptions_file, 'r') as file:
    for line in file:
        _, description = line.strip().split(maxsplit=1)
        if len(description.split()) > max_length:
            max_length = len(description.split())
        tokens = torchtext.data.utils.get_tokenizer("basic_english")(description.lower())
        counter.update(tokens)

# Create a vocabulary from the counter
specials = ['<unk>', '<pad>', '<bos>', '<eos>','endseq','startseq']
vocab_object = torchtext.vocab.Vocab(counter, specials=specials)

# Convert the vocab to a list of words
vocab = list(vocab_object.stoi.keys())
print(vocab)
print("Vocabulary size:", len(vocab))
print('Max Description Length:', max_length)


['<unk>', '<pad>', '<bos>', '<eos>', 'endseq', 'startseq', 'the', 'is', 'on', 'of', 'and', 'are', 'there', 'in', 'white', 'with', 'black', 'has', 'to', 'wearing', 'man', 'it', 'blue', 'brown', 'red', 'people', 'two', 'green', 'sitting', 'his', 'front', 'top', 'behind', 'one', 'standing', 'that', 'he', 'at', 'large', 'next', 'yellow', 'table', 'color', 'sky', 'small', 'an', 'some', 'building', 'side', 'be', 'other', 'shirt', 'wall', 'water', 'woman', 'person', 'street', 'this', 'from', 'have', 'can', 'cat', 'up', 'above', 'also', 'ground', 'light', 'trees', 'holding', 'left', 'fence', 'hair', 'her', 'many', 'right', 'them', 'background', 'down', 'seen', 'bed', 'plate', 'beside', 'by', 'gray', 'looking', 'sidewalk', 'sign', 'they', 'all', 'as', 'dark', 'grass', 'outside', 'she', 'walking', 'wooden', 'clock', 'long', 'road', 'room', 'their', 'three', 'back', 'baseball', 'counter', 'distance', 'parked', 'photo', 'pole', 'silver', 'sits', 'under', 'wood', 'around', 'dirt', 'head', 'metal', 

In [14]:
def load_clean_descriptions(filename):
    descriptions = []
    max_length = 0  # Initialize max_length variable
    with open(filename, 'r') as file:
        for line in file:
            _, description = line.strip().split(maxsplit=1)
            desc =  description # Add startseq and endseq
            descriptions.append(desc)
            # Update max_length if the length of the current description is greater
            if len(desc.split()) > max_length:
                max_length = len(desc.split())
    # Save the max_length to a file
    with open('max_description_length.txt', 'w') as max_length_file:
        max_length_file.write(str(max_length))
    return descriptions, max_length

train_descriptions, max_length = load_clean_descriptions('filtered_train_descriptions.txt')
print('Descriptions: train=%d' % len(train_descriptions))
print('Max Description Length:', max_length)


Descriptions: train=2000
Max Description Length: 240


In [2]:
print(train_descriptions)

['startseq yellow wicker and metal chair sits in corner of deck on boat there are backpacks along the side of the wooden deck shadows are casted on the floor rope ladder is to the left of the deck water surrounds the boat and island land masses are visible in the distance endseq', 'startseq group of people are at the beach two women are sitting in blue beach chairs they have blonde hair pink shirts shorts and are wearing sunglasses man is sitting behind them at picnic table in denim pants and blue and white stripped shirts shirtless boy is pushing bike along side young man four small children play near the shore the two boys are shirtless and wearing shorts and the girls are wearing bathing suits endseq', 'startseq two brown horses are standing together outside there heads are up against the side of the building the both have saddles on they are standing on concrete endseq', 'startseq this is picture of an outdoor scene there is river running horizontally through the picture with grass

In [1]:
import nltk

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def extract_predicates(captions):
    predicates = []
    for caption in captions:
        # Tokenize the caption
        tokens = nltk.word_tokenize(caption)
        
        # Perform POS tagging
        tagged = nltk.pos_tag(tokens)
        
        # Extract predicates (verbs)
        predicates_in_caption = [word for word, tag in tagged if tag.startswith('VB')]
        
        predicates.extend(predicates_in_caption)
    
    return predicates


predicates = extract_predicates(print(train_descriptions))
print(predicates)

ModuleNotFoundError: No module named 'nltk'

In [15]:
def read_max_description_length(filename='max_description_length.txt'):
    try:
        with open(filename, 'r') as file:
            max_length = int(file.read().strip())
        return max_length
    except FileNotFoundError:
        print("Max length file not found. Returning default value of 0.")
        return 0

# Usage example:
max_length = read_max_description_length()
print("Max Description Length:", max_length)


Max Description Length: 240


In [16]:
from collections import Counter
def create_vocabulary(descriptions):
    word_counter = Counter()
    for desc in descriptions:
        words = desc.lower().split()
        word_counter.update(words)
    # Add special tokens if needed
    special_tokens = ['<start>', '<end>', '<unk>']
    vocab = special_tokens + [word for word in word_counter if word_counter[word] > 0]
    return vocab

# Create vocabulary
vocab = create_vocabulary(train_descriptions)

# Print vocabulary
print("Vocabulary:", vocab)
print("Vocabulary Size:", len(vocab))

Vocabulary: ['<start>', '<end>', '<unk>', 'yellow', 'wicker', 'and', 'metal', 'chair', 'sits', 'in', 'corner', 'of', 'deck', 'on', 'boat', 'there', 'are', 'backpacks', 'along', 'the', 'side', 'wooden', 'shadows', 'casted', 'floor', 'rope', 'ladder', 'is', 'to', 'left', 'water', 'surrounds', 'island', 'land', 'masses', 'visible', 'distance', 'group', 'people', 'at', 'beach', 'two', 'women', 'sitting', 'blue', 'chairs', 'they', 'have', 'blonde', 'hair', 'pink', 'shirts', 'shorts', 'wearing', 'sunglasses', 'man', 'behind', 'them', 'picnic', 'table', 'denim', 'pants', 'white', 'stripped', 'shirtless', 'boy', 'pushing', 'bike', 'young', 'four', 'small', 'children', 'play', 'near', 'shore', 'boys', 'girls', 'bathing', 'suits', 'brown', 'horses', 'standing', 'together', 'outside', 'heads', 'up', 'against', 'building', 'both', 'saddles', 'concrete', 'this', 'picture', 'an', 'outdoor', 'scene', 'river', 'running', 'horizontally', 'through', 'with', 'grass', 'foreground', 'trees', 'background', 

In [18]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [24]:
import pickle
# File path where you want to save the data
file_path1 = "VG_train_wordtoix.pkl"

# Writing data to a file in binary mode
with open(file_path1, "wb") as f:
    pickle.dump(wordtoix, f)

import pickle
# File path where you want to save the data
file_path2 = "VG_train_ixtoword.pkl"

# Writing data to a file in binary mode
with open(file_path2, "wb") as f:
    pickle.dump(ixtoword, f)    

In [20]:
from torchtext.vocab import GloVe

def load_glove_embeddings(glove_dim=200):
    glove = GloVe(name='6B', dim=glove_dim)
    embeddings_index = {}
    
    for word, tensor in zip(glove.itos, glove.vectors):
        embeddings_index[word] = tensor.numpy()
        
    print('Found %s word vectors.' % len(embeddings_index))
    return embeddings_index

embeddings_index = load_glove_embeddings()


Found 400000 word vectors.


In [21]:
embedding_dim = 200
vocab_size = len(ixtoword)+1
# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [22]:
print(embedding_matrix.shape)
#print(embedding_matrix)
pickle_file = "train_embedding_matrix.pkl"

# Save the preprocessed data to a pickle file
with open(pickle_file, 'wb') as f:
    pickle.dump(embedding_matrix, f)

print("Preprocessed descriptions saved to:", pickle_file)

(4808, 200)
Preprocessed descriptions saved to: train_embedding_matrix.pkl


In [23]:
# Function to preprocess a single description
def preprocess_description(description):
        # Encode the sequence
    seq = [wordtoix[word] for word in description.split(' ') if word in wordtoix]
    # Split one sequence into multiple X, y pairs
    for i in range(1, len(seq)):
        # Split into input and output pair
        in_seq, out_seq = seq[:i], seq[i]
        # Pad input sequence
        in_seq += [0] * (max_length - len(in_seq))  # Padding with zeros
        in_seq = torch.tensor(in_seq)
        # Encode output sequence (one-hot encoding)
        out_seq_onehot = torch.zeros(vocab_size)  # Initialize all zeros
        out_seq_onehot[out_seq] = 1
    return in_seq, out_seq_onehot

# Sample descriptions file format: image_name description
descriptions_file = "filtered_train_descriptions.txt"
preprocessed_data = {}
# Read descriptions file line by line
with open(descriptions_file, 'r') as file:
    for line in file:
        image_name, description = line.strip().split(maxsplit=1)
        in_seq, out_seq_onehot = preprocess_description(description)
        print(out_seq_onehot)
        preprocessed_data[image_name] = {'X_pair': in_seq, 'y_pair': out_seq_onehot}

# Define the filename for saving the preprocessed data
pickle_file = "preprocessed_filtered_train_descriptions.pkl"

# Save the preprocessed data to a pickle file
with open(pickle_file, 'wb') as f:
    pickle.dump(preprocessed_data, f)

print("Preprocessed descriptions saved to:", pickle_file)

tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ...,

In [1]:
import json

# Read the JSON file
with open('paragraphs_v1.json', 'r') as f:
    data = json.load(f)

# Open the text file for writing
with open('paragraphs.txt', 'w') as f:
    # Write the header
    f.write("image,caption\n")
    
    # Iterate over each item in the JSON data
    for item in data:
        # Extract image_id and paragraph
        image_id = str(item["image_id"]) + ".jpg"
        caption = item["paragraph"]
        
        # Write to the text file
        f.write(f"{image_id},{caption}\n")


In [3]:
import json
import csv

# Read the JSON file
with open('paragraphs_v1.json', 'r') as infile:
    data = json.load(infile)

# Write to CSV file
with open('paragraphs.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['image', 'caption'])  # Write header
    for item in data:
        image_id = str(item['image_id'])
        image_name = image_id + '.jpg'  # Assuming the image file extension is jpg
        paragraph = item['paragraph']
        writer.writerow([image_name, paragraph])


In [7]:
import json
import csv
import os
import shutil

# Function to get the object name considering both 'name' and 'names' keys
def get_name(entity):
    if 'name' in entity:
        return entity['name']
    elif 'names' in entity:
        return entity['names'][0] if entity['names'] else None
    else:
        return None

# Step 1: Read the JSON file to get image IDs
with open('train_split.json', 'r') as image_ids_file:
    image_ids_json = json.load(image_ids_file)
    image_names = image_ids_json

# Step 2: Parse the JSON file to get relationships
with open('D:\\WORK\\M.SC\\MSC_Project\\GitHub\\Dataset\\VG\\relationships.json', 'r') as relationships_file:
    relationships_json = json.load(relationships_file)

# Step 3: Read the JSON file and match the image IDs
matched_data = []
for item in relationships_json:
    image_id = item['image_id']
    if (image_id) in image_names:
        relationships = set()  # Use a set to store unique relationships for each image
        for relationship in item['relationships']:
            subject = get_name(relationship['subject'])
            predicate = relationship['predicate']
            obj = get_name(relationship['object'])
            relationships.add(f"{subject} {predicate} {obj}")
        combined_relationships = ', '.join(relationships)
        matched_data.append([str(image_id), combined_relationships])

# Step 4: Read the CSV file and match the image names
image_captions = {}  # Dictionary to store image captions
with open('paragraphs.csv', 'r', newline='', encoding='utf-8') as csv_file:
    reader = csv.reader(csv_file)
    next(reader)  # Skip header
    for row in reader:
        image_name = row[0].split('.')[0]  # Remove the file extension
        if int(image_name) in image_names:
            image_captions[image_name] = row[1]  # Store image caption

# Step 5: Combine data and create a new CSV file with matched data
combined_matched_data = []
for image_id, relationships in matched_data:
    caption = image_captions[image_id]
    image_name1 = str(image_id) + '.jpg'
    combined_matched_data.append([image_name1, caption, relationships])

with open('train.csv', 'w', newline='', encoding='utf-8') as train_csv:
    writer = csv.writer(train_csv)
    writer.writerow(['image', 'caption', 'relationships'])  # Write header
    writer.writerows(combined_matched_data)

# Step 6: Copy matched images to a different folder
source_folder = 'D:\\WORK\\M.SC\\MSC_Project\\GitHub\\Dataset\\VG\\VG_100K\\'
destination_folder = 'D:\\WORK\\M.SC\\MSC_Project\\\\GitHub\\Dataset\\VG\\VG_train\\'

# Create the destination folder if it does not exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Copy matched images
for image_name, _, _ in combined_matched_data:
    #image_name = image_id + '.jpg'  # Assuming the image names are in the format "image_id.jpg"
    source_path = os.path.join(source_folder, image_name)
    destination_path = os.path.join(destination_folder, image_name)
    shutil.copyfile(source_path, destination_path)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\EXTREME_RED\\Downloads\\GitHub\\Dataset\\VG\\relationships.json'

In [6]:
# Step 1: Read the first file and store its content in a dictionary
file1_data = {}
with open( 'captions.txt' , "r") as file1:
    for line in file1:
        image_name, caption = line.strip().split(",", 1)
        if image_name in file1_data:
            file1_data[image_name].append(caption)
        else:
            file1_data[image_name] = [caption]

# Step 2: Read the second file and store its image file names in a set
file2_image_names = set()
with open('D:\\WORK\\M.SC\\MSC_Project\\GitHub\\Dataset\\Flickr_8k_testImages.txt', "r") as file2:
    for line in file2:
        image_name = line.strip()
        file2_image_names.add(image_name)

# Step 3-5: Write common image names and their captions to a new file
with open('test_captions.txt', "w") as new_file:
    for image_name in file2_image_names:
        if image_name in file1_data:
            captions = file1_data[image_name]
            for caption in captions:
                new_file.write(f"{image_name},{caption}\n")
